Q3. Train a model with autolog

We will train a RandomForestRegressor (from Scikit-Learn) on the taxi dataset.

We have prepared the training script train.py for this exercise, which can be also found in the folder homework.

The script will:

    load the datasets produced by the previous step,
    train the model on the training set,
    calculate the RMSE score on the validation set.

Your task is to modify the script to enable autologging with MLflow, execute the script and then launch the MLflow UI to check that the experiment run was properly tracked.

Tip 1: don't forget to wrap the training code with a with mlflow.start_run(): statement as we showed in the videos.

Tip 2: don't modify the hyperparameters of the model to make sure that the training will finish quickly.

What is the value of the min_samples_split parameter:

    2
    4
    8
    10


In [1]:
# Run the training script
!python train.py

RMSE: 5.431162180141208


In [2]:
# get the value of the min_samples_split parameter from the logged parameters
import mlflow

experiment = mlflow.get_experiment_by_name("Default")
run = mlflow.search_runs(experiment_ids=experiment.experiment_id)
min_samples_split = run['params.min_samples_split'][0]
print(f"min_samples_split: {min_samples_split}")

min_samples_split: 2


Q4. Launch the tracking server locally

Now we want to manage the entire lifecycle of our ML model. In this step, you'll need to launch a tracking server. This way we will also have access to the model registry.

Your task is to:

    launch the tracking server on your local machine,
    select a SQLite db for the backend store and a folder called artifacts for the artifacts store.

You should keep the tracking server running to work on the next two exercises that use the server.

In addition to backend-store-uri, what else do you need to pass to properly configure the server?

    default-artifact-root
    serve-artifacts
    artifacts-only
    artifacts-destination


Answer: default-artifact-root

Q5. Tune model hyperparameters

Now let's try to reduce the validation error by tuning the hyperparameters of the RandomForestRegressor using hyperopt. We have prepared the script hpo.py for this exercise.

Your task is to modify the script hpo.py and make sure that the validation RMSE is logged to the tracking server for each run of the hyperparameter optimization (you will need to add a few lines of code to the objective function) and run the script without passing any parameters.

After that, open UI and explore the runs from the experiment called random-forest-hyperopt to answer the question below.

Note: Don't use autologging for this exercise.

The idea is to just log the information that you need to answer the question below, including:

    the list of hyperparameters that are passed to the objective function during the optimization,
    the RMSE obtained on the validation set (February 2023 data).

What's the best validation RMSE that you got?

    4.817
    5.335
    5.818
    6.336


In [3]:
# Run the hyperparameter optimization script
!python hpo.py

2025/05/30 16:24:44 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-hyperopt' does not exist. Creating a new experiment.
🏃 View run amusing-dog-356 at: http://127.0.0.1:5000/#/experiments/549583683325628373/runs/21829d7a2738451d885dee867009ea25

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549583683325628373   

🏃 View run upset-kite-477 at: http://127.0.0.1:5000/#/experiments/549583683325628373/runs/cdffa2c43b284806916661a2a9441acf

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549583683325628373   

🏃 View run clean-crab-346 at: http://127.0.0.1:5000/#/experiments/549583683325628373/runs/18b99d8b47ab4bceac6dc8ae32fe6549

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549583683325628373   

🏃 View run marvelous-bug-890 at: http://127.0.0.1:5000/#/experiments/549583683325628373/runs/d1e0ba64d13e475a8b296eb05c9893bf

🧪 View experiment at: http://127.0.0.1:5000/#/experiments/549583683325628373   

🏃 View run trusting-bat-859 at: http:/

In [6]:
# Find the best validation RMSE from the logged runs
import mlflow

experiment = mlflow.get_experiment_by_name("random-forest-hyperopt")
runs = mlflow.search_runs(experiment_ids=experiment.experiment_id)
best_rmse = runs['metrics.validation_rmse'].min()
print(f"Best validation RMSE: {best_rmse:.3f}")

Best validation RMSE: 5.335


In [ ]:
experiments = mlflow.search_experiments()
for exp in experiments:
    print(
        f"Experiment ID: {exp.experiment_id}, Name: {exp.name}, Artifact Location: {exp.artifact_location}")

Q6. Promote the best model to the model registry

The results from the hyperparameter optimization are quite good. So, we can assume that we are ready to test some of these models in production. In this exercise, you'll promote the best model to the model registry. We have prepared a script called register_model.py, which will check the results from the previous step and select the top 5 runs. After that, it will calculate the RMSE of those models on the test set (March 2023 data) and save the results to a new experiment called random-forest-best-models.

Your task is to update the script register_model.py so that it selects the model with the lowest RMSE on the test set and registers it to the model registry.

Tip 1: you can use the method search_runs from the MlflowClient to get the model with the lowest RMSE,

Tip 2: to register the model you can use the method mlflow.register_model and you will need to pass the right model_uri in the form of a string that looks like this: "runs:/<RUN_ID>/model", and the name of the model (make sure to choose a good one!).

What is the test RMSE of the best model?

    5.060
    5.567
    6.061
    6.568



In [7]:
# Run the script to register the best model
!python register_model.py

2025/05/30 16:28:24 INFO mlflow.tracking.fluent: Experiment with name 'random-forest-best-models' does not exist. Creating a new experiment.
🏃 View run upset-mouse-756 at: http://127.0.0.1:5000/#/experiments/679606160065087578/runs/192f671a6e5a40439ae33dd3f91a60f7
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/679606160065087578
🏃 View run unruly-koi-142 at: http://127.0.0.1:5000/#/experiments/679606160065087578/runs/a92f3768958a484db03bcfd7e7e28d10
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/679606160065087578
🏃 View run gifted-zebra-804 at: http://127.0.0.1:5000/#/experiments/679606160065087578/runs/edcccbc022d74292890be40b9ff7500e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/679606160065087578
🏃 View run nebulous-foal-560 at: http://127.0.0.1:5000/#/experiments/679606160065087578/runs/5cdff8ffab154fc5b93b51e6e43b82e2
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/679606160065087578
🏃 View run nervous-mole-490 at: http://127.0.0.1:5000

In [8]:
# Check what is the best model's RMSE
import mlflow

experiment = mlflow.get_experiment_by_name("random-forest-best-models")
runs = mlflow.search_runs(experiment_ids=experiment.experiment_id)
best_rmse = runs['metrics.test_rmse'].min()
print(f"Best test RMSE: {best_rmse:.3f}")

Best test RMSE: 5.567
